<a href="https://colab.research.google.com/github/andrewcharles/air.signal/blob/main/earth_engine/check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# A notebook for checking the latest sentinel images for a defined area.
# First, need a means of storing previous imagery.

In [10]:
from google.colab import auth
auth.authenticate_user()

In [11]:
from google.colab import drive
drive.mount('/content/drive')
from google.oauth2 import service_account
from google.cloud import storage

# Replace <path-to-json-key-file> with the path to your JSON key file
credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/earth-engine-workflow-013a80bf4f5b.json')
service_account = 'harvester@earth-engine-workflow.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/content/drive/MyDrive/earth-engine-workflow-013a80bf4f5b.json')
ee.Initialize(credentials)
client = storage.Client(credentials=credentials)
client
OUTPUT_BUCKET = 'airsignal2023'

def create_bucket(bucket_name):
    storage_client = storage.Client()    
    bucket = storage_client.create_bucket(bucket_name)
    print('Bucket {} created'.format(bucket.name))

def bucket_exists(bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    try:
        bucket.reload()
        return True
    except Exception as e:
        print(e)
        return False

# Replace <BUCKET_NAME> with the name of your bucket
if bucket_exists(OUTPUT_BUCKET):
  print('Bucket exists')
else:
  print('Bucket does not exist')
  create_bucket(OUTPUT_BUCKET)


Mounted at /content/drive
Bucket exists


In [9]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=k-e_UeNZgFOGCH_3IH4TdVLHKiM1FuH5W9uqNfm3Y_g&tc=u9bxS93GL2ic5_uAyMjZWyPoPE2yO1HtLFjA10dcOvo&cc=P-wNmLvPKmsWESSGNeVv_GFvABs3UcZosutI7jRkn60

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk4bxFUbAjHnkLpuWJvEkGYdITdMGZ26FNT4gCYh8jCl6h9-kwU9vFY

Successfully saved authorization token.


In [ ]:
!pip install earthengine-api

In [12]:
# Anglesea Heath
xmin = 144.3
xmax = 144.25
ymin = -38.3
ymax = -38.44

ROI_POLY = ee.Geometry.Polygon([[[xmin, ymax],
                          [xmin, ymin],
                          [xmax, ymin],
                          [xmax, ymax]]])
lat,lon = (ymin+ymax)/2,(xmin+xmax)/2

In [13]:
# Define a first set of dates
START_DATE = "2023-01-01"
END_DATE =  "2023-02-01"

In [14]:
# SENTINEL FUNCTIONS
def se2mask(image):
    quality_band = image.select('QA60')
    # using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11    
    # we only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0)) 
    # we'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask)#.divide(10000)

def scale(image):
  bands = ['B2','B3','B4']
  im2 = image.select(bands).multiply(0.0001)
  image = image.addBands(im2.select(bands))
  return image

In [15]:
BANDS = ['B1','B2','B3','B4','B8','QA60']
CLOUD_LIMIT = 50
imcol = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(ROI_POLY).filterDate(
    START_DATE, END_DATE).filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",CLOUD_LIMIT)).select(BANDS).map(scale)
#
#.map(se2mask)
#map(lambda img: img.divide(10000))

In [16]:
# Some information about the image list we are processing
imlist = imcol.toList(imcol.size())
unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")) 
print(ee.List.getInfo(unique_dates))

['2023-01-05', '2023-01-05', '2023-01-10', '2023-01-10', '2023-01-15', '2023-01-15', '2023-01-25', '2023-01-25']


In [192]:
def get_name(img):
  return(ee.Image(img).getString('id'))

In [19]:
unique_ids = imlist.map(lambda im: ee.Image(im).id()  )
idslist = ee.List.getInfo(unique_ids)

In [3]:
# STORAGE BUCKET
PROJECT_ID = "earth-engine-workflow" #@param {type:"string"}
BUCKET_NAME = "airsignal2023" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}
USER_NAME = 'charlesan'
# Cloud Storage bucket into which  
# datasets will be written.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'airsignal2023'

In [22]:
import json
json.dumps(idslist)

'["20230105T002049_20230105T002432_T54HYC", "20230105T002049_20230105T002432_T55HBT", "20230110T002051_20230110T002415_T54HYC", "20230110T002051_20230110T002415_T55HBT", "20230115T002049_20230115T002358_T54HYC", "20230115T002049_20230115T002358_T55HBT", "20230125T002049_20230125T002320_T54HYC", "20230125T002049_20230125T002320_T55HBT"]'

In [24]:
client = storage.Client(credentials=credentials)
BUCKET = client.get_bucket(OUTPUT_BUCKET)
filename = "idlist.json"
blob = BUCKET.blob(filename)
json_object = idslist

# upload the blob 
blob.upload_from_string(
    data=json.dumps(json_object),
    content_type='application/json'
)
result = filename + ' upload complete'

In [169]:
img = imcol.first().clip(roi_poly)
names = img.bandNames()
bands = ['B2','B3','B4']
im2 = img.select(bands).multiply(0.0001)
image = img.addBands(im2.select(bands))

In [170]:
names = img.bandNames()

In [171]:
ee.List.getInfo(names)

['B1', 'B2', 'B3', 'B4', 'B8', 'QA60']

In [114]:
# NB this can be expensive to run!
print(ee.Number.getInfo(imcol.size()))

12


In [127]:
range = imcol.reduceColumns(ee.Reducer.minMax(), ['system:time_start'])
ee.Dictionary.getInfo(range)

{'max': 1674606455028, 'min': 1673310454396}

In [133]:
#his function adds a band representing the image timestamp.
def addTime(image):
  return image.addBands(image.getNumber('system:time_start'))

#Map the function over the collection and display the result.
imcol = imcol.map(addTime)

In [154]:
img = imcol.max().clip(roi_poly).select(['B2','B3','B4']).divide(1000) #RGB

In [144]:
img

In [177]:
# Test image
viz = {'bands':['B2','B3','B4'], 'min': 0, 'max': 1, 'dimensions': 512}
thumb_url = img.getThumbURL(viz)
print(thumb_url)
from IPython.display import Image
Image(url=thumb_url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/5740df384fade6e3505e51e7078ecec9-0f115f1c100686ab2c12f6b6980c93fe:getPixels


In [105]:
def mosaicByDate(imcol):
  imlist = imcol.toList(imcol.size())
  unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")).distinct()

  def match_dates(d):
    d = ee.Date(d)
    dateString = ee.Date(d).format('yyyy-MM-dd')
    im = imcol.filterDate(d, d.advance(1, "day")).mosaic()
    return im.set(
        "system:time_start", d.millis(), 
        "system:id", d.format("YYYY-MM-dd")).rename(dateString)

  mosaic_imlist = unique_dates.map(match_dates)

  return ee.ImageCollection(mosaic_imlist)

moscol = mosaicByDate(imcol)

In [110]:
moscol

In [ ]:
unique_dates = moscol.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")) 
#print(ee.List.getInfo(unique_dates))

In [109]:
unique_dates

In [ ]:
# Save this image collection

In [ ]:
# Define another date
# This time search the local archives to see if these dates are present

In [1]:
%%capture
#!pip install geemap

In [3]:
#import numpy as np
#
#import geemap
#import pandas as pd
#import plotnine
#from scipy import stats
#import tensorflow as tf
#print(tf.__version__)
#print(folium.__version__)

In [ ]:
import geemap

In [ ]:
#imgs = imgs.map(addNDVI)
imgs = addNDVI(imgs)

In [ ]:
rgb = ['B4','B3','B2','B8']
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}
ndviViz = {"min":0.0, "max":1.0,"bands":['NDVI']}

In [ ]:
# initialize our map
map1 = geemap.Map(height=400,width=600)
map1.setCenter((xmin+xmax)/2, (ymin+ymax)/2,  14)
map1.addLayer(imgs.clip(roi_poly), ndviViz, "NDVI")
map1.addLayerControl()
map1

Map(center=[-38.209999999999994, 144.4175], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [ ]:
imgs = imgs.select(['B4','B3','B2','B8'])

In [ ]:
# how to download https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api#export_a_geotiff_file
task = ee.batch.Export.image.toDrive(**{
  'image': imgs,
  'description': 'reedy',
  'folder':'CNN',
  'region': roi_poly, 
  'crs': 'EPSG:7854', 
  'maxPixels': 1e13})
task.start()